In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import string
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

In [2]:
company_names = pd.read_csv('../Company Names.csv')
company_names

<IPython.core.display.Javascript object>

,Company
0,2U INC
1,30DC INC
2,3D PIONEER SYSTEMS INC
3,8X8 INC
4,A10 NETWORKS INC
...,...
287,ZOOM VIDEO COMUNICATIONS INC
288,ZOOMAWAY TRAVEL INC
289,ZOOMINFO TECHNOLOGIES
290,ZSCALER INC


In [3]:
columns_cik_numbers = ['Company','CIK','NaN']
cik_numbers = pd.read_csv('../CIK.txt',sep = ':',header = None,names = columns_cik_numbers,low_memory = False)
cik_numbers = cik_numbers.drop(columns='NaN')
cik_numbers.head()

<IPython.core.display.Javascript object>

,Company,CIK
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607
2,#1 ARIZONA DISCOUNT PROPERTIES LLC,0001457512
3,#1 PAINTBALL CORP,0001433777
4,$ LLC,0001427189


In [4]:
punctuations = string.punctuation
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).upper())

company_names['Company'] = company_names['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
company_names['Company'] = company_names['Company'].apply(lambda x: str(x).upper())

required_data = pd.merge(company_names,cik_numbers,on = 'Company')
required_data['CIK'] = required_data['CIK'].apply(lambda x: str(int(x)))
required_data = required_data.drop_duplicates(subset=['Company'],ignore_index=True)

required_data

<IPython.core.display.Javascript object>

,Company,CIK
0,2U INC,1459417
1,30DC INC,1118974
2,3D PIONEER SYSTEMS INC,1556753
3,8X8 INC,1023731
4,A10 NETWORKS INC,1580808
...,...,...
237,ZIX CORP,855612
238,ZOOM TELEPHONICS INC,822708
239,ZOOMAWAY TRAVEL INC,1541054
240,ZSCALER INC,1713683


In [5]:
manual_data = {'ADVANCED VOICE RECOGNITION':'1342936','AMERICAN SECURITY RES CORP':'1085069',
              'AMERICAN SOFTWARE  CL A':'713425','AUTOMATIC DATA PROCESSING':'8670',
              'AVAYA HLDGS CORP':'1418100','BIGCOMMERCE HOLDIN INC':'1626450',
              'BIOKEY INTERNATIONAL INC':'1242388','BLAQCLOUDS INC':'1517990',
              'BSQUARE CORP':'1054721','BUSYBOX COM INC':'1088316',
              'CERIDIAN HCM HOLDING':'1725057','CHANGSHENG INTERNATIONAL GRO':'1451797',
              'CHINA YANYUN YHU NTL ED GRP':'1773086','CLOUDWARD INC':'1045739',
              'COM GUARDCOM INC':'1103759','DUCK CREEK TECHNOL INC':'1160951',
              'FANTASY ACES DAILY FANTASY':'1653115','FLEXSHARES IBOXX 3YR TAR FD':'',
              'FLEXSHARES IBOXX 5YR TAR FD':'','GTY TECHNOLOGY HOLDINGS':'1682325',
              'HRSOFT INC':'917019','INTEGRATED BUSINESS SYS SVC':'1040863',
              'INTELLIGENT SYSTEM CORP':'320340','INTERNATIONAL LEADERS CAP CO':'1470550',
              'ISHARES IBOXX HIGH YLD CP BD':'','ISHARES IBOXX INVST GR CP BD':'',
              'KIWIBOX COM INC':'838796','LIQUI BOX CORP':'216430',
              'LIVE MICROSYSTEMS INC':'915866','MEDALLIANCE INC':'886037',
              'MEDIATECHNICS CORP':'','MONSTER ARTS':'1423746',
              'PALANTIR TECHNOLOG INC':'1321655','PROGRESS SOFTWARE CORP':'876167',
              'QUANTGATE SYSTEMS INC':'','RORINE INTL HOLDING CORP':'1423586',
              'SAILPOINT TECHNO HLDG':'1627857','SKYBOX INTL INC':'909687',
              'SSC TECHNOLOGIES HLDGS INC':'878390','SYNCHRONOSS TECHNOLOGIES':'1131554',
              'TERADATA CORP':'320045','THEGLOBECOM INC':'1066684',
              'VANCORD CAPITAL INC':'1200528','VDOPH INTERNATIONAL INC':'1499355',
              'VERB TECHNOLOGY CO INC':'1566610','VERTICAL COMPUTER SYS INC':'1099509',
              'VMWARE INC CL A':'1124610','WOD RETAIL SOLUTIONS':'704366',
              'ZOOM VIDEO COMUNICATIONS INC':'1585521','ZOOMINFO TECHNOLOGIES':'1794515'}
manual_data = {'Company':list(manual_data.keys()),'CIK':list(manual_data.values())}
manual_data = pd.DataFrame(manual_data)
required_data = pd.concat([required_data,manual_data],ignore_index = True)
required_data

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Company,CIK
0,2U INC,1459417
1,30DC INC,1118974
2,3D PIONEER SYSTEMS INC,1556753
3,8X8 INC,1023731
4,A10 NETWORKS INC,1580808
...,...,...
287,VERTICAL COMPUTER SYS INC,1099509
288,VMWARE INC CL A,1124610
289,WOD RETAIL SOLUTIONS,704366
290,ZOOM VIDEO COMUNICATIONS INC,1585521


In [6]:
required_data[required_data['CIK'] == '']

,Company,CIK
259,FLEXSHARES IBOXX 3YR TAR FD,
260,FLEXSHARES IBOXX 5YR TAR FD,
266,ISHARES IBOXX HIGH YLD CP BD,
267,ISHARES IBOXX INVST GR CP BD,
272,MEDIATECHNICS CORP,
276,QUANTGATE SYSTEMS INC,


In [9]:

for company,cik in zip(required_data['Company'][41:100],required_data['CIK'][41:100]): #change indices as mentioned
    
    os.makedirs('../Data/{}/10-K'.format(company))
    os.makedirs('../Data/{}/10-Q'.format(company))
    os.makedirs('../Data/{}/8-K'.format(company))
    i = 1
    while True:
        try:
            driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
            driver.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

            time.sleep(1)

            driver.find_element(By.XPATH,"//div[@id='filingsStart']/div[2]/div[3]/h5/a").click()

            view_button = driver.find_element(By.XPATH,"//button[@data-group='annualOrQuarterlyReports']")
            driver.implicitly_wait(1)
            ActionChains(driver).move_to_element(view_button).click(view_button).perform()
            
            driver.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']").clear()
            from_date = driver.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']")
            from_date.send_keys('2018-01-01')
            
            driver.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']").clear()
            to_date = driver.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']")
            to_date.send_keys('2021-12-31')
            to_date.send_keys(Keys.ENTER)
            time.sleep(1)
            
        except:
            break
            
        try :
            
            form_type = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[1]".format(i)).text
            href = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[2]/div/a[2]".format(i)).get_attribute('href')
            i += 1
            if form_type not in ['10-K','10-Q','8-K']:
                continue
            
        except:
            break
        
        
        try :
            options = webdriver.ChromeOptions()
            download_dir = f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}"
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs",prefs)
            driver_10s = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe",options = options)
            driver_10s.get(href)
            
        except:
            continue
            
        acc_number = driver_10s.find_element(By.XPATH,"//div[@id='secNum']").text.split('.')[-1].strip()
        acc_number = ''.join(acc_number.split('-'))
        xlsx_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_number}/Financial_Report.xlsx"
        driver_10s.get(xlsx_href)
        time.sleep(4)
        driver_10s.close()
        try:
            os.rename(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}\\Financial_Report.xlsx",
                     f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}\\{acc_number}.xlsx")
        except:
            continue
    
    driver.close()
    i = 1
    while True:
        try:
            driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
            driver.get(f"https://www.sec.gov/edgar/browse/?CIK={cik}&owner=exclude")

            time.sleep(1)

            driver.find_element(By.XPATH,"//div[@id='filingsStart']/div[2]/div[2]/h5/a").click()

            view_button = driver.find_element(By.XPATH,"//button[@data-group='currentReports']")
            driver.implicitly_wait(1)
            ActionChains(driver).move_to_element(view_button).click(view_button).perform()
            
            driver.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']").clear()
            from_date = driver.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']")
            from_date.send_keys('2018-01-01')
            
            driver.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']").clear()
            to_date = driver.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']")
            to_date.send_keys('2021-12-31')
            to_date.send_keys(Keys.ENTER)
            time.sleep(1)
        except:
            break
            
        try :
            
            form_type = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[1]".format(i)).text
            href = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[2]/div/a[2]".format(i)).get_attribute('href')
            i += 1
            if form_type not in ['10-K','10-Q','8-K']:
                continue
            
        except:
            break
        
        
        try :
            
            options = webdriver.ChromeOptions()
            download_dir = f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}"
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs",prefs)
            driver_8k = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe",options = options)
            driver_8k.get(href)
        except:
            continue
            
        acc_number = driver_8k.find_element(By.XPATH,"//div[@id='secNum']").text.split('.')[-1].strip()
        acc_number = ''.join(acc_number.split('-'))
        xlsx_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_number}/Financial_Report.xlsx"
        driver_8k.get(xlsx_href)
        time.sleep(4)
        driver_8k.close()
        try:
            os.rename(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}\\Financial_Report.xlsx",
                      f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\{form_type}\\{acc_number}.xlsx")
        except:
            continue
    driver.close()
     

In [31]:
bad_companies = set()
for i,(dirpath,dirnames,filenames) in enumerate(os.walk('../Data')):
        
    if len(dirnames) == 0 and len(filenames) == 0:
        bad_companies.add(str(dirpath).split('\\')[-2])

In [33]:
len(bad_companies)

80

In [34]:
bad_companies = required_data[required_data['Company'].isin(bad_companies)]
bad_companies = bad_companies.reset_index().drop(columns = ['index'])
bad_companies

,Company,CIK
0,30DC INC,1118974
1,3D PIONEER SYSTEMS INC,1556753
2,ACTUA CORP,1085621
3,ADEXA INC,1119691
4,ADVANTE CORP,1407847
...,...,...
75,REALPAGE INC,1286225
76,RIGHTSCORP INC,1506270
77,RINGCENTRAL INC,1384905
78,ROSETTA STONE INC,1351285


In [38]:
for company,cik in zip(bad_companies['Company'][13:],bad_companies['CIK'][13:]):
    
    i = 1
    while True:
        try:
            options = webdriver.ChromeOptions()
            download_dir = f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K"
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs",prefs)
            driver_10K = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe",options = options)
            driver_10K.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

            time.sleep(3)

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='Search table']").clear()
            form_type = driver_10K.find_element(By.XPATH,"//input[@placeholder='Search table']")
            form_type.send_keys('10-K')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']").clear()
            from_date = driver_10K.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']")
            from_date.send_keys('2010-01-01')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']").clear()
            to_date = driver_10K.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']")
            to_date.send_keys('2021-12-31')
            to_date.send_keys(Keys.ENTER)
            time.sleep(1)

            driver_10K.find_element(By.XPATH,"//input[@id='colAccNumber']").click()
            time.sleep(1)
            form_type = driver_10K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[1]").text
            if form_type not in ['10-K','10-Q','8-K']:
                i += 1
                continue
            
    
        except:
            break
    
    
        try :
            acc_number = driver_10K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[5]").text.strip()
            acc_number = ''.join(acc_number.split('-'))
            xlsx_href = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_number}/Financial_Report.xlsx'
            driver_10K.get(xlsx_href)
            time.sleep(4)
            i += 1
            driver_10K.close()
            
        except:
            continue
        
        try:
            os.rename(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K\\Financial_Report.xlsx",
                     f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K\\{acc_number}.xlsx")
        except:
#             os.remove(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K\\Financial_Report.xlsx")
            continue

    i = 1
    driver_10K.close()
    while True:
        try:
            options = webdriver.ChromeOptions()
            download_dir = f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-Q"
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs",prefs)
            driver_10Q = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe",options = options)
            driver_10Q.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

            time.sleep(3)

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='Search table']").clear()
            form_type = driver_10Q.find_element(By.XPATH,"//input[@placeholder='Search table']")
            form_type.send_keys('10-Q')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']").clear()
            from_date = driver_10Q.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']")
            from_date.send_keys('2010-01-01')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']").clear()
            to_date = driver_10Q.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']")
            to_date.send_keys('2021-12-31')
            to_date.send_keys(Keys.ENTER)
            time.sleep(1)

            driver_10Q.find_element(By.XPATH,"//input[@id='colAccNumber']").click()
            time.sleep(1)
            form_type = driver_10Q.find_element(By.XPATH,f"//tbody/tr[{i}]/td[1]").text
            if form_type not in ['10-K','10-Q','8-K']:
                i += 1
                continue
            
            
        except:
            break
    
    
        try :
            acc_number = driver_10Q.find_element(By.XPATH,f"//tbody/tr[{i}]/td[5]").text.strip()
            acc_number = ''.join(acc_number.split('-'))
            xlsx_href = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_number}/Financial_Report.xlsx'
            driver_10Q.get(xlsx_href)
            time.sleep(4)
            i += 1
            driver_10Q.close()
            
        except:
            continue
        
        try:
            os.rename(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-Q\\Financial_Report.xlsx",
                     f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-Q\\{acc_number}.xlsx")
        except:
#             os.remove(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K\\Financial_Report.xlsx")
            continue

    i = 1
    driver_10Q.close()
    while True:
        try:
            options = webdriver.ChromeOptions()
            download_dir = f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\8-K"
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs",prefs)
            driver_8K = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe",options = options)
            driver_8K.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

            time.sleep(3)

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='Search table']").clear()
            form_type = driver_8K.find_element(By.XPATH,"//input[@placeholder='Search table']")
            form_type.send_keys('8-K')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']").clear()
            from_date = driver_8K.find_element(By.XPATH,"//input[@placeholder='From Date (yyyy-mm-dd)']")
            from_date.send_keys('2010-01-01')

    #         driver_10K.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']").clear()
            to_date = driver_8K.find_element(By.XPATH,"//input[@placeholder='To Date (yyyy-mm-dd)']")
            to_date.send_keys('2021-12-31')
            to_date.send_keys(Keys.ENTER)
            time.sleep(1)

            driver_8K.find_element(By.XPATH,"//input[@id='colAccNumber']").click()
            time.sleep(1)
            form_type = driver_8K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[1]").text
            if form_type not in ['10-K','10-Q','8-K']:
                i += 1
                continue
            
            
        except:
            break
    
    
        try :
            acc_number = driver_8K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[5]").text.strip()
            acc_number = ''.join(acc_number.split('-'))
            xlsx_href = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_number}/Financial_Report.xlsx'
            driver_8K.get(xlsx_href)
            time.sleep(4)
            i += 1
            driver_8K.close()
            
        except:
            continue
        
        try:
            os.rename(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\8-K\\Financial_Report.xlsx",
                     f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\8-K\\{acc_number}.xlsx")
        except:
#             os.remove(f"C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{company}\\10-K\\Financial_Report.xlsx")
            continue

    i = 1
    driver_8K.close()